# Analyse av kode
### Intro
I denne notebooken trekker vi fram essensen i main_app.py og forklarer hva som skjer
#### Hoved punkter
- Henter data
- Renser data
- Visualserer historisk data
- Visualiserer prediktiv data
- En knapp som setter det sammen

In [6]:
import os
import sys


project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

import streamlit as st
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
import plotly.io as pio
import nbformat
pio.renderers.default = "notebook_connected"

from src.data_collection.frost import FrostAPI
from src.visualization.plot_frost import DataPlotting
from src.visualization.plot_predictive import PlottingPredictiveAnalysis
from src.analysis.data_analysis import DataAnalysis



## Henting av data
her så bruker vi FrostAPI klassen til å hente data for en gitt tidsperiode, 

(denne koden representere det som skjer i "fetch_data_time" funskjonen)

In [13]:
api = FrostAPI()
start = (datetime.now() - timedelta(days=100)).strftime('%Y-%m-%d')
end = datetime.now().strftime('%Y-%m-%d')
df = api.fetch_data_for_periode(start, end, intervall="W")

Henter data fra 2025-02-15 til 2025-02-22
Henter data fra 2025-02-22 til 2025-03-01
Henter data fra 2025-03-01 til 2025-03-08
Henter data fra 2025-03-08 til 2025-03-15
Henter data fra 2025-03-15 til 2025-03-22
Henter data fra 2025-03-22 til 2025-03-29
Henter data fra 2025-03-29 til 2025-04-05
Henter data fra 2025-04-05 til 2025-04-12
Henter data fra 2025-04-12 til 2025-04-19
Henter data fra 2025-04-19 til 2025-04-26
Henter data fra 2025-04-26 til 2025-05-03
Henter data fra 2025-05-03 til 2025-05-10
Henter data fra 2025-05-10 til 2025-05-17
Henter data fra 2025-05-17 til 2025-05-24
Henter data fra 2025-05-24 til 2025-05-26


## Data rensing
her bruker vi dataen vi hentet i forrige bolk "hent data", denne Dataframen sender vi inn i DataAnalysis klassen.
Deretter ville vi kjørt en for-løkke som itererer igjennom alle kolonnene, men nå viser vi bare et eksempel med temeratur.
Her får vi ut 4 dataframes, disse returneres i prosjektet som en Dictionary.

(denne koden representerer det som skje i "clean_data" funskjonen)

In [ ]:

data_analyzer = DataAnalysis(df)
cleaned_data, outliers = data_analyzer.remove_outliers("temperature")
statistics = data_analyzer.calculate_all_statistics()
correlation = data_analyzer.analyse_correlation()

print(cleaned_data)

                             stasjon  temperature  cloud_area_fraction  \
tidspunkt                                                                
2025-02-15 00:00:00+00:00  SN18700:0         -4.6                  NaN   
2025-02-15 00:00:00+00:00  SN90450:0         -4.8                  NaN   
2025-02-15 00:00:00+00:00  SN69100:0         -1.7                  4.0   
2025-02-15 00:00:00+00:00  SN50539:0          1.2                  NaN   
2025-02-15 01:00:00+00:00  SN18700:0         -4.6                  NaN   
...                              ...          ...                  ...   
2025-05-25 22:00:00+00:00  SN18700:0         11.2                  NaN   
2025-05-25 23:00:00+00:00  SN50539:0          9.9                  NaN   
2025-05-25 23:00:00+00:00  SN18700:0         10.9                  NaN   
2025-05-25 23:00:00+00:00  SN69100:0         13.0                  8.0   
2025-05-25 23:00:00+00:00  SN90450:0          9.6                  NaN   

                           wind_speed

## Plotting av historisk data
her bruker vi den rensede dataen vi fikk fra funksjonen Clean_data, denne dataen sender vi inn i DataPlotting, for å så representere plottingen
ved hjelp av funskjonene i DataPlotting, det finnes flere grafer enn vist under, og i main_app kjører vi en for-løkke for å iterere igjennom alle kolonnene.
(denne koden representerer "plot_historical_data")

In [ ]:
analyzer = DataPlotting(cleaned_data)
fig_1 = analyzer.plot_histogram("temperature", "Temperatur")
fig_2 = analyzer.plot_timeseries("temperature", "Temperatur")
if fig_1 is not None:
    fig_1.show()
else:
    print("fig_1 er tom")
if fig_2 is not None:
    fig_2.show()
else:
    print("fig_2 er tom")



## Plotting av predikert data
Her så sender vi inn start å slutt dato, PlottingPredictiveAnalysis klassen bruker da disse datoene til å trene opp en prediskjonmodell ved hjelp av PredictiveAnalysis, vi plotter så denne presikjonsmodellen og sammenlikner den med de faktiske dataene. Deretter bruker vi denne prediskjonsmodellen til å predikere været, og plotter dette ved hjelp av PlottingPredictiveAnalysis klassen.

In [14]:
plotter = PlottingPredictiveAnalysis(start, end)
prediction = plotter.analysis

fig_3 = plotter.plot_predictive_analysis()
if fig_3 is not None:
    fig_3.show()
else:
    print("fig_3 er tom")


history = df["temperature"].iloc[-20:]
forecast = prediction.forecast(days=7)
fig_4 = plotter.plot_forecast(history, forecast)

if fig_4 is not None:
    fig_4.show()
else:
    print("fig_4 er tom")



Henter data fra 2025-02-15 til 2025-03-15
Henter data fra 2025-03-15 til 2025-04-15
Henter data fra 2025-04-15 til 2025-05-15
Henter data fra 2025-05-15 til 2025-05-26
Raw df_periode rows: 9600
Unique stations: ['SN18700:0' 'SN50539:0' 'SN69100:0' 'SN90450:0']


## Oppsumering
Dette er main_app.py skrellet ned til sine helt grunnlegende komponenter, main_app bruker streamlit til å systematisere alle plottene samt at vi iterere igjennom flere kolonner og bruker flere forskjellige grafer. Alikevel koker prosjektet ned til bolkene over, nemlig:
- Henter data
- Renser data
- Visualserer historisk data
- Prediker data
- Visualiserer prediktiv data
som er vist over.
